# Árboles AVL
Un **árbol AVL** es un árbol binario de búsqueda auto-balanceado donde para cada nodo:
balance = altura(subárbol_izq) - altura(subárbol_der) ∈ {-1, 0, 1}.
Se re-balancea mediante **rotaciones** tras inserciones y eliminaciones.
Rotaciones:
- Simple derecha (LL)
- Simple izquierda (RR)
- Doble izquierda-derecha (LR)
- Doble derecha-izquierda (RL)
Objetivo: mantener operaciones en O(log n).

## Implementación básica
Definimos Nodo y Árbol AVL con utilidades de depuración.

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Any, Iterable, List

@dataclass
class Node:
    key: Any
    left: Optional['Node'] = None
    right: Optional['Node'] = None
    height: int = 1  # altura del nodo (hoja = 1)

class AVLTree:
    def __init__(self):
        self.root: Optional[Node] = None

    # --- Utilidades de altura y balance ---
    def _height(self, n: Optional[Node]) -> int:
        return n.height if n else 0

    def _update(self, n: Node):
        n.height = 1 + max(self._height(n.left), self._height(n.right))

    def _balance_factor(self, n: Optional[Node]) -> int:
        return self._height(n.left) - self._height(n.right) if n else 0

    # --- Rotaciones ---
    def _rotate_right(self, y: Node) -> Node:
        x = y.left
        T2 = x.right if x else None
        # rotación
        x.right = y
        y.left = T2
        # actualizar alturas
        self._update(y)
        self._update(x)
        return x

    def _rotate_left(self, x: Node) -> Node:
        y = x.right
        T2 = y.left if y else None
        # rotación
        y.left = x
        x.right = T2
        # actualizar alturas
        self._update(x)
        self._update(y)
        return y

    # --- Rebalanceo ---
    def _rebalance(self, node: Node) -> Node:
        self._update(node)
        balance = self._balance_factor(node)
        # Caso LL
        if balance > 1 and self._balance_factor(node.left) >= 0:
            return self._rotate_right(node)
        # Caso RR
        if balance < -1 and self._balance_factor(node.right) <= 0:
            return self._rotate_left(node)
        # Caso LR
        if balance > 1 and self._balance_factor(node.left) < 0:
            node.left = self._rotate_left(node.left)
            return self._rotate_right(node)
        # Caso RL
        if balance < -1 and self._balance_factor(node.right) > 0:
            node.right = self._rotate_right(node.right)
            return self._rotate_left(node)
        return node

    # --- Inserción ---
    def insert(self, key: Any):
        def _insert(node: Optional[Node], key: Any) -> Node:
            if not node:
                return Node(key)
            if key < node.key:
                node.left = _insert(node.left, key)
            elif key > node.key:
                node.right = _insert(node.right, key)
            else:
                return node  # ignorar duplicados
            return self._rebalance(node)
        self.root = _insert(self.root, key)

    # --- Nodo mínimo ---
    def _min_node(self, node: Node) -> Node:
        while node.left:
            node = node.left
        return node

    # --- Eliminación ---
    def delete(self, key: Any):
        def _delete(node: Optional[Node], key: Any) -> Optional[Node]:
            if not node:
                return None
            if key < node.key:
                node.left = _delete(node.left, key)
            elif key > node.key:
                node.right = _delete(node.right, key)
            else:
                # encontrado
                if not node.left and not node.right:
                    return None
                if not node.left:
                    return node.right
                if not node.right:
                    return node.left
                # dos hijos
                succ = self._min_node(node.right)
                node.key = succ.key
                node.right = _delete(node.right, succ.key)
            return self._rebalance(node) if node else None
        self.root = _delete(self.root, key)

    # --- Búsqueda ---
    def search(self, key: Any) -> bool:
        cur = self.root
        while cur:
            if key == cur.key:
                return True
            cur = cur.left if key < cur.key else cur.right
        return False

    # --- Recorridos ---
    def inorder(self) -> List[Any]:
        res = []
        def _in(n: Optional[Node]):
            if not n: return
            _in(n.left); res.append(n.key); _in(n.right)
        _in(self.root)
        return res

    # --- Visualización simple ---
    def pretty(self):
        lines = []
        def _print(node: Optional[Node], prefix="", is_left=True):
            if not node:
                lines.append(prefix + ("└── " if is_left else "┌── ") + "·")
                return
            lines.append(prefix + ("└── " if is_left else "┌── ") + f"{node.key} (h={node.height}, b={self._balance_factor(node)})")
            if node.left or node.right:
                _print(node.left, prefix + ("    " if is_left else "│   "), True)
                _print(node.right, prefix + ("    " if is_left else "│   "), False)
        _print(self.root)
        return "\n".join(lines)

def build_avl(seq: Iterable[Any]) -> AVLTree:
    t = AVLTree()
    for x in seq:
        t.insert(x)
    return t

## Ejemplos de rotaciones
Forzamos cada caso con una secuencia concreta.

In [ ]:
def demo_rotaciones():
    casos = {
        'LL (simple derecha)': [30, 20, 10],          # desequilibrio a la izquierda
        'RR (simple izquierda)': [10, 20, 30],         # desequilibrio a la derecha
        'LR (doble izquierda-derecha)': [30, 10, 20],  # primero izquierda luego derecha
        'RL (doble derecha-izquierda)': [10, 30, 20],  # primero derecha luego izquierda
    }
    for nombre, seq in casos.items():
        t = build_avl(seq)
        print(f'Caso: {nombre} - inserciones: {seq}')
        print(t.pretty())
        print('-'*60)

demo_rotaciones()

## Ejemplo completo de inserciones
Secuencia más larga y verificación del in-order (ordenado).

In [ ]:
seq = [50, 20, 70, 10, 40, 60, 80, 30, 45, 42, 43]
t = build_avl(seq)
print('In-order:', t.inorder())
print(t.pretty())

## Eliminación
Probamos eliminar (1) hoja, (2) nodo con un hijo, (3) nodo con dos hijos.

In [ ]:
print('Árbol inicial:')
print(t.pretty())

# 1. Eliminar hoja (43)
t.delete(43)
print('\nTras eliminar hoja 43:')
print(t.pretty())

# 2. Eliminar nodo con un hijo (42)
t.delete(42)
print('\nTras eliminar nodo con un hijo 42:')
print(t.pretty())

# 3. Eliminar nodo con dos hijos (20)
t.delete(20)
print('\nTras eliminar nodo con dos hijos 20:')
print(t.pretty())
print('\nIn-order final:', t.inorder())

## Complejidad
Sea n el número de nodos.
- Altura del AVL: O(log n)
- Búsqueda: O(log n)
- Inserción: O(log n) (búsqueda + hasta 1 rebalanceo con ≤ 2 rotaciones)
- Eliminación: O(log n)
- Rotación simple o doble: O(1)
Espacio adicional: O(h) en recursión (≈ O(log n)).
Peor caso teórico (por implementación): coste constante de rotaciones y mantenimiento de alturas.

## Resumen
El árbol AVL garantiza equilibrio estricto manteniendo factor de balance en {-1,0,1}. Esto asegura altura O(log n) y operaciones eficientes incluso en secuencias adversas. Las rotaciones corrigen localmente el desequilibrio sin afectar el orden in-order.

## Cómo usar este notebook
1. Ejecuta las celdas en orden (Shift+Enter).
2. La clase `AVLTree` queda disponible tras la celda de implementación.
3. Usa `build_avl([valores])` para construir rápidamente un árbol.
4. Métodos clave:
- `insert(valor)` agrega y rebalancea.
- `delete(valor)` elimina y rebalancea.
- `search(valor)` devuelve True/False.
- `inorder()` devuelve los valores ordenados.
- `pretty()` devuelve representación textual del árbol.
5. Modifica las listas de ejemplo para provocar diferentes rotaciones.
6. Si algo falla (NameError), reinicia el kernel y vuelve a ejecutar todo.

In [ ]:
# Ejemplo rápido de uso independiente
demo = AVLTree()

for v in [15, 10, 20, 8, 12, 25, 16]:
    demo.insert(v)
print('Árbol inicial:')
print(demo.pretty())

print('\nBuscar 12 ->', demo.search(12))
print('Buscar 99 ->', demo.search(99))

demo.delete(10)  # elimina nodo con dos hijos (10 tiene 8 y 12)
print('\nTras eliminar 10:')
print(demo.pretty())

print('\nRecorrido in-order:', demo.inorder())